In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### **Section 1: Setup**

In [14]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [16]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("ADK component imported!!!")

ADK component imported!!!


In [17]:
retry_config=types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429,500,503,504]
)

### **Section 2: creation multi agent sys**
- research +summeriser team

In [20]:
# research agent
research_agent=Agent(
    name="ResearchAgent",
    model=Gemini(
        model='gemini-2.5-flash-lite',
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings"
)

#Summarizer Agent:
summarizer_agent= Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}. Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("worker Agents created!!")

worker Agents created!!


In [21]:
# root coordinator
root_agent=Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)]
)
print("root coordinator created!!")

root coordinator created!!


In [23]:
runner=InMemoryRunner(agent=root_agent)
response= await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > Here's a concise summary of the advancements in quantum computing and their implications for AI:

*   **Revolutionary Computational Power:** Quantum computers, using qubits and superposition, offer parallel processing capabilities far exceeding classical computers, enabling AI to handle massive datasets and complex problems more efficiently.
*   **Transformative AI Capabilities:** Quantum computing is set to enhance AI through faster data processing and pattern recognition, accelerated machine learning training (QML), and significantly improved optimization for tasks like drug discovery and logistics.
*   **Tackling Intractable Challenges & Hybrid Futures:** Quantum AI can solve problems currently impossible for classical computers and will likely be implemented through hybrid quantum-classical approaches, leveraging the strengths of both.
*   **Significant Investment and Future Impact:** Leading organizations like IBM and Google are investing heavily in quantum c

### *‼️ However, relying on an LLM's instructions to control the order can sometimes be unpredictable. Next, we'll explore a different pattern that gives you guaranteed, step-by-step execution.*